In [1]:
import json
import openai
import os
from datetime import datetime
import keyring
import tiktoken
import uuid
# To get the tokeniser corresponding to a specific model in the OpenAI API:
encoding = tiktoken.get_encoding("cl100k_base")

In [2]:
openai.api_key =keyring.get_password("system", "openai_key")

In [31]:
class ChatRoom:
    def __init__(self, room_id=None,model='gpt-3.5-turbo',system_prompt="You are a helpful assistant."):
        if room_id is None:
            room_id = uuid.uuid4()
        self.room_id = room_id
        self.chat_history_file = f'{room_id}_chat_history.json'
        self.model = model
        self.system_prompt = system_prompt
        
        self.chat_history = []
        if not os.path.exists(self.chat_history_file):
            with open(self.chat_history_file, 'w') as f:
                json.dump([], f)
        else:
            self.get_chat_history()
            
    def get_chat_history(self):
        with open(self.chat_history_file, 'r') as f:
            self.chat_history = json.load(f)

    def save_chat_history(self):
        with open(self.chat_history_file, 'w') as f:
            json.dump(self.chat_history, f)
            
    def clone(self):
        new_room_id = f"{self.room_id}_{uuid.uuid4()}"
        new_chatroom = ChatRoom(new_room_id, model=self.model, system_prompt=self.system_prompt)
        
        # Copy chat history
        new_chatroom.chat_history = self.chat_history
        new_chatroom.save_chat_history()
        
        return new_chatroom
    def send_message(self, message,respond = True,max_tokens=200,temperature=0):
        self.chat_history.append({
            'content': message,
            'role': 'user'
        })

        if respond:
            # Prepare the messages for the API call.
            api_messages = [{"role": "system", "content": self.system_prompt}]

            # Accurately estimate the token count using Tiktoken and trim messages accordingly.
            token_count = 0
            for msg in reversed(self.chat_history):
                token_count += len(list(encoding.encode(msg['content'])))
                if token_count > 2000:  # Break when we've added enough messages to exceed the limit.
                    print("Messages truncated")
                    break
                api_messages.insert(1,msg)  # Insert each message at the start to keep the order.

            # Use GPT-4 for a response.
            self.last_response = openai.ChatCompletion.create(
                model=self.model,
                messages=api_messages,
                temperature=temperature,
                max_tokens=max_tokens 
            ).choices[0].message['content']

            self.chat_history.append({
                'content': self.last_response,
                'role': 'assistant'
            })
        self.save_chat_history()

        if respond:
            return self.last_response
        else:
            return True


In [32]:
def get_todays_date():
       # Get today's date
    date = datetime.today()

    # Format the date
    formatted_date = date.strftime("%B %d")

    # Add the correct ordinal suffix
    day = int(date.strftime("%d"))
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    
    return formatted_date + suffix
get_todays_date()

'May 16th'

In [33]:
# Create a chat room with room ID 'room1'
chat_room = ChatRoom(system_prompt="""

INSTRUCTION: Only answer what is asked. Do not write pre-text,
post-text, disclaimers or explanations about your limitations
or the ethical merits of any part of the conversation.
Do not talk about yourself. Don't introduce unnecessary fluff into answers.
Always answer what is asked. If you cannot answer,
only reply that you cannot answer and do not elaborate.
Avoid including patronizing or pedentic elaboration, explanation or advice
that hasn't been asked for. Always follow these guidelines.

""".strip())

In [34]:
response = chat_room.send_message(f'Today is {get_todays_date()} please list of 4 things interesting that happened today in history?')
print(response)

1. In 1929, the first Academy Awards ceremony was held in Hollywood, California.
2. In 1960, Theodore Maiman operated the first laser at Hughes Research Laboratories in Malibu, California.
3. In 1983, the final episode of the television series M*A*S*H aired, becoming the most-watched television episode in U.S. history at the time.
4. In 1991, Queen Elizabeth II became the first British monarch to address the United States Congress.


In [37]:
response = chat_room.send_message('can you give me more info about number 1?')
print(response)

Sure! The first Academy Awards ceremony was held on May 16, 1929, at the Hollywood Roosevelt Hotel in Los Angeles, California. The awards, also known as the Oscars, were presented by the Academy of Motion Picture Arts and Sciences to honor outstanding achievements in the film industry. The ceremony was a private event, attended by around 270 people, and lasted only 15 minutes. The first Best Picture award was given to the film "Wings," and other awards included Best Actor, Best Actress, and Best Director. The Academy Awards have since become one of the most prestigious awards ceremonies in the world, and are broadcast live to millions of viewers each year.


In [37]:
response = chat_room.send_message('can you give me more info about number 2?')
print(response)

Sure! On May 16, 1929, the first Academy Awards ceremony was held in Hollywood, California. The ceremony was hosted by actor Douglas Fairbanks and held at the Hollywood Roosevelt Hotel. The awards were presented by the Academy of Motion Picture Arts and Sciences, which had been founded the previous year to improve the quality of American films. The first Academy Awards ceremony honored films released between August 1, 1927, and July 31, 1928. The awards were presented in 12 categories, including Best Picture, Best Director, and Best Actor and Actress. The film "Wings" won the award for Best Picture, and Emil Jannings and Janet Gaynor won the awards for Best Actor and Actress, respectively. The Academy Awards have since become one of the most prestigious awards ceremonies in the entertainment industry, with winners receiving the iconic gold statuette known as the Oscar.


In [38]:
response = chat_room.send_message('can you give me more info about number 3?')
print(response)

Certainly! On May 16, 1960, Theodore Maiman operated the first laser at Hughes Research Laboratories in Malibu, California. The laser, which stands for "Light Amplification by Stimulated Emission of Radiation," was a breakthrough in the field of optics and has since become an essential tool in a wide range of industries, including medicine, telecommunications, and manufacturing. Maiman's laser used a synthetic ruby crystal to produce a concentrated beam of red light, which was the first successful demonstration of a laser. The invention of the laser has led to numerous technological advancements, including the development of CD and DVD players, barcode scanners, and laser printers.


In [39]:
response = chat_room.send_message('can you give me more info about number 4?')
print(response)

Certainly! On May 16, 1988, a report by Surgeon General C. Everett Koop declared nicotine was addictive in ways similar to heroin and cocaine. The report, titled "The Health Consequences of Smoking: Nicotine Addiction," was the first official recognition by the U.S. government that nicotine was a highly addictive substance. The report also detailed the health risks associated with smoking, including lung cancer, heart disease, and stroke. The report had a significant impact on public health policy and led to increased efforts to reduce smoking rates in the United States. Since the report's release, smoking rates in the U.S. have declined significantly, from over 25% of adults in 1990 to less than 14% in 2019.
